In [4]:
import logfire
from pydantic_ai import Agent, RunContext

from config import config

ValidationError: 1 validation error for Config
logfire_token
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [8]:
agent_configs = [
    (
        'LogAgent',
        'Collect logs and give insightful reports about how the users are interactig with the site',
    ),
    (
        'MetricAgent',
        'Summarize the metric usage such as CPU, Memory, Disk, Network, etc. Detect anomalies in the system if any, and notify the team',
    ),
    (
        'RoulatteAgent',
        'Use the `roulette_wheel` function to see if the customer has won based on the number they provide.',
    ),
]
agents: dict[str, Agent] = {
    name: Agent(
        model='ollama:llama3.2',
        name=name,
        system_prompt=instruction,
    )
    for name, instruction in agent_configs
}


# Tools

These are functions that can be attached to an agent
similar to FastAPI views that can be attached to a a router

Agent can use tell the model to use these functions with the given context
and we can validate the output type

These are useful abstractions to organize code since the unpredictable runtime context can cause spagetti-code



In [9]:
roulette_agent = agents['RoulatteAgent']


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'


In [10]:
success_number = 18
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(result.data)

RuntimeError: This event loop is already running